<a href="https://colab.research.google.com/github/MaicolFranco/Prediction_Credit_Card/blob/main/Proyecto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
#importaciones correspondientes
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
plt.style.use("seaborn")

In [30]:
df = pd.read_csv("/content/BankChurners.csv")
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [27]:
df = df.drop(['CLIENTNUM', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1)


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

# **Data Source**
https://www.kaggle.com/

Dataset description
The dataset consists of 10127 observations, one for each different customer bank's account. For each account the provided relevant information are the following:

Attrition_Flag : Wether the user leaved or not the service (bank's account closed or not)
Customer_Age : Customer's age in year
Gender : Customer's gender (male or female)
Dependent_count : Number of people who depend upon the customer for their support and welfare.
Education_Level : Customer's educational qualification (high school, graduate, etc.)
Marital_Status : Customer's marital status (married, single, etc.)
Income_Category :Customer's income bracket in dollars (less than 40K, 40K-60K, etc.)
Card_Category : Credit card category (Blue, Silver, etc.)
Months_on_book : Period of relationship with bank in months
Total_Relationship_Count : Total number of products held by the customer
Months_Inactive_12_mon : Number of months inactive in the last 12 months
Contacts_Count_12_mon : Number of contacts (phone calls) in the last 12 months
Credit_Limit : Credit limit on the credit card
Total_Revolving_Bal : Total revolving balance on the credit card
Avg_Open_To_Buy : Open to buy credit line (average of last 12 months). This also turns out to be the difference between the credit limit (Credit_Limit) assigned to a cardholder account and the present balance on the account (Total_Revolving_Bal).
Total_Amt_Chng_Q4_Q1 : Change in total transactions amount (Q4 over Q1)
Total_Trans_Amt : Total transactions amount (last 12 months)
Total_Trans_Ct : Total number of transactions (last 12 months)
Total_Ct_Chng_Q4_Q1 : Change in total number of transactions (Q4 over Q1)
Avg_Utilization_Ratio : Average card utilization ratio


# Objetive:
get to know why some people renounce the services of the cards, we could also see if the stratum of the people influences their decisions

# classification or regression?
This proyect is clasification

# **features**
Card_Category'

Gender

Education_Level

Card_Category

In [32]:
df.shape

(10127, 23)

In [29]:
# df = pd.read_csv("/content/Electric Vehicles.csv")
# df.head(30)